### [Design a HashMap](https://leetcode.com/problems/design-hashmap/)

Design a HashMap without using any built-in hash table libraries.

To be specific, your design should include these functions:

put(key, value) : Insert a (key, value) pair into the HashMap. If the value already exists in the HashMap, update the value.
get(key): Returns the value to which the specified key is mapped, or -1 if this map contains no mapping for the key.
remove(key) : Remove the mapping for the value key if this map contains the mapping for the key.

**Example:**
```
MyHashMap hashMap = new MyHashMap();
hashMap.put(1, 1);          
hashMap.put(2, 2);         
hashMap.get(1);            // returns 1
hashMap.get(3);            // returns -1 (not found)
hashMap.put(2, 1);          // update the existing value
hashMap.get(2);            // returns 1 
hashMap.remove(2);          // remove the mapping for 2
hashMap.get(2);            // returns -1 (not found) 
```
**Note:**

- All keys and values will be in the range of [0, 1000000].
- The number of operations will be in the range of [1, 10000].
- Please do not use the built-in HashMap library.


In [1]:
class MyHashMap(object):
    def __init__(self):
        """
        Initialize your data structure here.
        """
        # we need something to store the keys
        # given size of the hashmap: 0...1,000,000
        # number of operations: 1...10000.. could e put/get/remove
        
        # for a hashmap, we need to hash the key on each operation
        #   put - compute the hash.. if the hash is not in the list, create a new entry
        #                            if the hash is already in the list, update the value.
        #   get - compute the has.. if the hash is in the list, we return the value.. else we return -1
        #   remove - compute the hash..remove the key from the list only if it exists. 
        #
        # what hashing function to use?
        #   key space - 1...1000000
        # ideally to avoid collision completely, we can pre-allocate a list of 1M entries. 
        # that would certainly be expensive in terms of storage
        #
        # since the number of operations are in the range of 1..10000
        # what hashing function to use? the result of the hashing function is used to directly
        # index the key in our list. 
        #   mod % 10000??.. if there is a collision, we append to the value list
        #   and do linear search of key (linear probing)
        #   our storage is indexed by the hash value..and each index points to a list of (key, value)
        #   where we perform linear search get/put/remove
        # with a storage of 10K, each index will grow up to 100 entries in the worst case. 
        #   as an optimization..we could store the list in a sorted way.. so retrieval can be done in log(n) instead of O(n)
        # is there a way to insert in sorted order in python? we could have our own insertion sort.
        #
        # what if we start out with 1000 or 10000 and then grow? we can double the size
        # on each collision.. since our hashing function changes upon change in the map size,
        # we have to recompute the hash of all existing keys
        #   
        
        self.MAP_SIZE = 5000 # Tried out different MAP sizes..1000, 5000, 10000, 50000
        self.hashmap = [[]] * self.MAP_SIZE # List of [key, value] list 
        self.KEY_NOT_FOUND = -1
    
    def hash(self, key):
        return (key % self.MAP_SIZE)

    def put(self, key, value):
        """
        value will always be non-negative.
        :type key: int
        :type value: int
        :rtype: void
        """
        keyhash = self.hash(key)
        
        
        # update the existing value if it exists
        keyfound = False
        for index, (k, v) in enumerate(self.hashmap[keyhash]):
            if k == key:
                # update the new value 
                self.hashmap[keyhash][index][1] = value
                keyfound = True
                break

        if not keyfound:
            # add it our list
            self.hashmap[keyhash].append([key, value])
            
        # Tried our sorting on insertion so that we can 
        # retrive from the retrieve faster on get.
        # but it didn't yield signficant boost in performance
        # with the dataset in the OJ
        # self.hashmap[keyhash].sort(key=lambda x: x[0])

    
    def binsearch(self, keys, key):
        """ an utility function to find the key in a sorted list of [key, value] items"""
        low = 0
        high = len(keys) - 1
        
        while low <= high:
            mid = (low + high) // 2
            k, _ = keys[mid]
            if key < k:
                high = mid - 1
            elif key > k:
                low = mid + 1
            else:
                # found the key we are looking for
                # return the index
                return mid
        
        return self.KEY_NOT_FOUND
            

    def get(self, key):
        """
        Returns the value to which the specified key is mapped, or -1 if this map contains no mapping for the key
        :type key: int
        :rtype: int
        """
        keyhash = self.hash(key)
        
        # doing linear search for now
        for k, v in self.hashmap[keyhash]:
            if k == key:
                return v
        
        return self.KEY_NOT_FOUND
    
        # can be replaced with binary search of key
        # run time of this problem with linear search in get() came around 7.8 secs as per OJ
        # lets try bin search
        
#         index = self.binsearch(self.hashmap[keyhash], key)
        
#         if index != self.KEY_NOT_FOUND:
#             # return the value
#             return self.hashmap[keyhash][index][1]
                    
#         return self.KEY_NOT_FOUND
        

    def remove(self, key):
        """
        Removes the mapping of the specified value key if this map contains a mapping for the key
        :type key: int
        :rtype: void
        """
        keyhash = self.hash(key)
        
        for index, (k, v) in enumerate(self.hashmap[keyhash]):
            if k == key:
                del self.hashmap[keyhash][index]
                return
        
        # Deletion using binary search
        # index = self.binsearch(self.hashmap[keyhash], key)
        # if index != self.KEY_NOT_FOUND:
        #     del self.hashmap[keyhash][index]


# Your MyHashMap object will be instantiated and called as such:
# obj = MyHashMap()
# obj.put(key,value)
# param_2 = obj.get(key)
# obj.remove(key)